In [132]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import time

import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import pos_tag, word_tokenize

from gensim.summarization.summarizer import summarize
from gensim import corpora
from gensim.models import LdaModel, LdaMulticore

from GoogleNews import GoogleNews
googlenews = GoogleNews()

import newspaper
from newspaper import Article
from newspaper import fulltext


In [51]:
#Блок выгрузки новостей из гугла
#новости выгружаются по заданной теме, глубина выгрузки - 10 страниц

TOPIC = 'China'
gnews_links = []
gnews=[]
googlenews.search(TOPIC)

#замеряем время
start_time = time.time()

for i in range(1,11):
    googlenews.clear()
    googlenews.getpage(i)
    for j in range(0, len(googlenews.gettext())):
        gnews.append(googlenews.gettext()[j])
        gnews_links.append(googlenews.getlinks()[j])

print("--- На формирование массива затрачено %s секунд ---" % (time.time() - start_time))             
    

'NoneType' object has no attribute 'get'
'NoneType' object has no attribute 'get'


["Can Hong Kong's fighting spirit endure? An answer might lie ...",
 'Second Pacific nation in a week ditches Taiwan for Beijing',
 'Asian stocks higher as India slashes taxes, China cuts new ...',
 "'The era of economic surrender is over,' says Mike Pence on ...",
 'China depends on foreign oil. The Saudi attack is a wake-up call',
 'Light shows and live bands: China wants to boost its nightlife ...',
 "The story of China's great corporate sell-off",
 'China Launches 5 New Earth Observation Satellites into Orbit ...',
 "China's younger citizens are reviving Imperial-era garments ...",
 'Trump grants tariff exemptions to plastic straws, dog leashes ...',
 "Why China isn't as skillful at disinformation as Russia",
 'China confirms detention of American FedEx pilot suspected ...',
 "China's Lunar Rover Scopes Out Weird Substance on Far ...",
 'China to tap pork reserves as swine fever hits industry',
 'European markets edge higher as US-China trade talks ...',
 'Asian markets rise as US,

In [79]:
#блок выгрузки новостей и формирования массива текстов
body = []
count = 0
error_position = [] #массив с номерами битых ссылок

#замеряем время
start_time = time.time()

for url in gnews_links:
    try:
        html = requests.get(url).text
        text = fulltext(html)
        body.append(text)
    except:
        error_position.append(count) #иногда попадаются битые ссылки. Здесь мы будем сохранять их позиции
        pass
    count += 1
    
print("--- На выгрузку затрачено %s секунд ---" % (time.time() - start_time))             


In [152]:
#Блок предобработки текста

stop_words = stopwords.words('english')
stop_words = stop_words + ['china', 'chinese','china\'s','china’s'] #в стоп список включаем слова, которые являются перекрестными в разных темах
lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stop_words, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in words if item not in stop]
    words = [item for item in words if len(item)>3]
    
    return words#' '.join(words) #здесь выбираем формат вывода: в виде отдельных токенов или в форме связной строки-текста

In [153]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in body:
    prepared_texts.append(prepare_text(item))

In [155]:
# Create the Inputs of LDA model: Dictionary and Corpus

dct = corpora.Dictionary(prepared_texts)
corpus = [dct.doc2bow(line) for line in prepared_texts]


In [156]:
#Train the LDA model

lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=7,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

# save the model
lda_model.save('lda_model.model')

In [165]:
# See the topics
lda_model.print_topics(-1)

[(0,
  '0.002*"coal" + 0.002*"diplomatic" + 0.002*"state" + 0.002*"said" + 0.002*"would" + 0.002*"year" + 0.002*"taiwan" + 0.002*"great" + 0.002*"government" + 0.002*"solomon"'),
 (1,
  '0.002*"system" + 0.002*"million" + 0.002*"would" + 0.002*"company" + 0.002*"said" + 0.002*"beijing" + 0.002*"airport" + 0.002*"dalai" + 0.001*"said." + 0.001*"pork"'),
 (2,
  '0.005*"hong" + 0.003*"kong" + 0.002*"party" + 0.002*"image" + 0.002*"uighur" + 0.002*"national" + 0.002*"mayila" + 0.002*"political" + 0.002*"lunar" + 0.002*"yutu-2"'),
 (3,
  '0.006*"said" + 0.004*"trade" + 0.004*"would" + 0.004*"u.s." + 0.004*"could" + 0.003*"according" + 0.003*"hong" + 0.003*"also" + 0.003*"bank" + 0.003*"last"'),
 (4,
  '0.003*"serbia" + 0.001*"serbian" + 0.001*"smederevo," + 0.001*"belgrade" + 0.001*"steel" + 0.001*"mill" + 0.001*"embassy" + 0.001*"european" + 0.001*"posted" + 0.001*"year"'),
 (5,
  '0.000*"climate" + 0.000*"awareness" + 0.000*"campaign" + 0.000*"hong" + 0.000*"information" + 0.000*"picture"

In [2]:
#Выгружаем все свежие заголовки про Китай
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#r1 = requests.get('https://www.reuters.com/search/news?blob=metals')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h2', class_='FeedItemHeadline_headline')

#Выводим заголовки
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')

Sri Lanka presidential nominee Rajapaksa would restore relations with China: adviser
///
China sovereign fund CIC says cautious of U.S. investment amid trade war
///
U.S., Chinese trade deputies face off in Washington amid deep differences
///
Singapore luxury apartment sales hit 11-year high, driven by Chinese demand
///
Taiwan cuts ties with Kiribati amid China tension
///
Cloud and clear: Tencent draws on gaming tech to lift B2B cloud ambitions
///
Breakingviews - Beijing’s monetary easing path is a conflicted one
///
China to set stricter targets for polluting cities in heating season
///
Hanfu movement sweeps China in revival of traditional culture
///
China green lights eight Argentine meat plants for beef exports
///
Explainer: U.S.-China trade talks - where they are and what's at stake
///
Factbox: Tariffs in place or planned for nearly all goods traded by U.S. and China
///
Federer happy for China to host Laver Cup
///
Chinese officials to visit U.S. farmland as trade talks co

In [3]:
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


--- 16.95953917503357 seconds ---


In [4]:
#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

In [5]:
#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))


In [13]:
#задаем ключевые слова по темам
topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR','trade']
topics_metals = ['metal', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']
print(summarize(prepared_texts[0], word_count=60))


“sessions agriculture get disproportionate amount air time,” source said, adding one sessions also include focus u.s. president donald trump’s demand china cut shipments synthetic opioid fentanyl united states trump eager provide export opportunities u.s. farmers, one key political constituencies battered china’s retaliatory tariffs u.s. soybeans agricultural commodities.


In [19]:
# отбор новостей по ключевым словам
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_econ:
            counter += 1
    if counter > 2:
        print('Economic story %s', i)
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1


Economic story %s 0
* gold slips, dollar gains ground * oil prices cool saudi production pledge (updates late afternoon) stephen culp new york, sept 18 (reuters) - u.s. stocks extended losses u.s. treasury yield curve flattened wednesday, u.s. federal reserve cut interest rates, expected, gave mixed signals regarding future rate moves.
Ссылка на статью: https://www.reuters.com/article/idUSL3N26A106
///


### Блок металлов

In [18]:
#Выгружаем все свежие заголовки про металлы из рейтерс
# s = requests.session()
# s.cookies.clear()

#r1 = requests.get('https://www.reuters.com/places/china', headers={'Cache-Control': 'no-cache'})
#https://www.reuters.com/news/archive/metals-news
r1 = requests.get('https://www.reuters.com/search/news?blob=Metals&sortBy=date&dateRange=all')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('h3', class_='search-result-title')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(coverpage_news_r[i].get_text())
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 3 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


ЗАГОЛОВКИ СТАТЕЙ
********************
PRECIOUS-Gold little changed after Fed outlook disappoints
///
Large parts of Bermuda plunged into darkness as Hurricane Humberto whips island
///
UPDATE 4-Large parts of Bermuda plunged into darkness as Hurricane Humberto whips island
///
Australian shares track regional gains after Fed cut; NZ up
///
Stocks fall, Treasury yield curve flattens after Fed cuts rates but sends mixed signals
///
GLOBAL MARKETS-Stocks fall, Treasury yield curve flattens after Fed cuts rates but sends mixed signals
///
PRECIOUS-Gold slides 1% after Fed cuts rates, but policy outlook uncertain
///
Exclusive: London Metal Exchange to delay ban on tainted metal until 2025 - sources
///
CANADA STOCKS-TSX pressured by drop in energy, materials sectors
///
PRECIOUS-Gold holds tight range as investors await Fed verdict
///
--- 6.705703258514404 seconds ---
********************
РЕЗЮМЕ СТАТЕЙ
********************
Metals story 0
* boj leaves interest rates unchanged * gold target

In [17]:
#Парсинг статей про металлы из архива
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('https://www.reuters.com/news/archive/metals-news')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('div', class_='story-content')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = 'https://www.reuters.com'+coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='StandardArticleBody_body')
    try:
        x = body[0].find_all('p')
    except:
        pass
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


ЗАГОЛОВКИ СТАТЕЙ
********************



PRECIOUS-Gold little changed after Fed outlook disappoints

        
        * Gold targets $1,446/oz - technicals
(Adds comments, details and updates prices)

12:44am EDT


///



GLOBAL MARKETS-Asian shares turn lower on guarded Fed, yen rises after BOJ

        
        Asian shares turned lower on
Thursday after the U.S. Federal Reserve cut interest rates as
expected but signalled a higher bar to further policy easings.

Sep 18 2019


///



PRECIOUS-Gold edges lower after Fed delivers mixed signals

        
        Gold prices edged lower on Thursday as
investors remained cautious after the U.S. Federal Reserve cut
interest rates as expected but delivered mixed signals about its
next move.

Sep 18 2019


///



GLOBAL MARKETS-Asian shares edge higher after Fed, investors await BOJ

        
        * BOJ seen on hold, but other central banks are easing
policy

Sep 18 2019


///



UPDATE 1-U.S. trade groups urge Congress to rein in 'Tariff

In [213]:
#Парсинг статей про металлы с сайта Kitco
# s = requests.session()
# s.cookies.clear()

r1 = requests.get('http://www.kitcometals.com/news/')
coverpage =  r1.content
soup = BeautifulSoup(coverpage, 'html5lib')
coverpage_news_r = soup.find_all('td', class_='text')

#Выводим заголовки
print('ЗАГОЛОВКИ СТАТЕЙ')
print('********************')
for i in range(0, len(coverpage_news_r)):
    print(re.sub('\t','',coverpage_news_r[i].get_text()))    #таким образом убираем тупые пробелы перед заглавием
    print('///')
    
# Парсим все статьи по существующим заголовкам
number_of_articles = len(coverpage_news_r)
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

#замеряем время
start_time = time.time()

#формируем массив с корпусами статей
for n in np.arange(0, number_of_articles):
    
    
    # Getting the link of the article
    link = coverpage_news_r[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news_r[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('article', itemprop='articleBody')
    try:
        x = body[0].find_all('p')
    except:
        pass

    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)
print("--- %s seconds ---" % (time.time() - start_time))


#Блок предобработки текста
from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

lemmatizator = nltk.stem.WordNetLemmatizer()
tokenizer = WhitespaceTokenizer()


#функция предобработки
def prepare_text(text, stop = stopwords_en, tokenizer = tokenizer):
    """ 
    Возвращает тексты: 
        * лемматизированные,
        * без стоп-слов, 
        * в нижнем регистре, 
        * все слова длиннее 3 символов

    text: string
        Текст поста

    parameters: list 
        stop: список из стоп-слов, example: ['а', политик', 'выбирать']
        tokenizer: токенизатор для текста, поданного на вход
    """
    tokens = tokenizer.tokenize(text.lower())
    #words = [lemmatizator.lemmatize(token) for token in tokens]
    words = [item for item in tokens if item not in stop]
    
    return ' '.join(words)

#Проходим по корпусу выгруженных новостей и выполняем предобработку всего корпуса
prepared_texts = []
for item in news_contents:
    prepared_texts.append(prepare_text(item))

topics_econ = ['economy', 'inflation', 'cpi', 'gdp', 'output', 'pboc', 'central bank', 'money market', 'RRR']
topics_metals = ['base', 'industrial', 'precious', 'metals', 'mining', 'coal', 'ore', 'iron', 'gold', 'platinum', 'copper', 'silver']

print('********************')
print('РЕЗЮМЕ СТАТЕЙ')
print('********************')
previous_item = ''
i = 0
for item in prepared_texts:
    counter = 0
    for word in item.split(' '):
        if word in topics_metals:
            counter += 1
    if counter > 1 and item != previous_item:
        print('Metals story %s' %(i))
        print(summarize(item, word_count=60))
        print('Ссылка на статью:', list_links[i])
        print('///')
        i += 1
    previous_item = item


ЗАГОЛОВКИ СТАТЕЙ
********************

Gold, silver prices back off a bit, but more upside likely   
- Kitco News, Sep 17 2019  7:59AM
///

Copper and other metals ease ahead of U.S.-China talks 
- Reuters, Sep 17 2019  7:25AM
///

Could more consolidation in the mining sector be on the way? 
- Kitco News, Sep 16 2019  7:43PM
///

After this happens, junior mining stocks will finally breakout to the upside 
- Kitco News, Sep 16 2019  7:40PM
///

Miners are still hungry for junior acquisitions 
- Kitco Video News, Sep 16 2019  7:26PM
///

Now is "absolutely" the buying time for junior miners 
- Kitco Video News, Sep 16 2019 12:08PM
///

Hedge funds dump gold, but bears are still hiding 
- Kitco News, Sep 16 2019 12:19PM
///

Trade talks seen as unlikely to mend U.S.-China divide 
- Reuters, Sep 17 2019  9:37AM
///

North-Alaska Pogo mine gets $30 million expansion 
- Kitco News, Sep 16 2019  2:57PM
///

Hedge funds turned bullish on oil before Saudi attacks: Kemp 
- Reuters, Sep 17 2019

--- 96.62188482284546 seconds ---
********************
РЕЗЮМЕ СТАТЕЙ
********************
Metals story 0
(kitco news) - gold silver prices modestly weaker early u.s. trading tuesday, corrective pressure following monday’s gains.
bears' next near-term downside price breakout objective pushing december futures prices solid technical support $1,485.00.
wyckoff's market rating: 7.0 december silver futures bulls still overall near-term technical advantage.
silver bulls' next upside price breakout objective closing prices solid technical resistance last week’s high $18.555 ounce.
Ссылка на статью: https://www.kitco.com/news/2019-09-17/Gold-silver-prices-back-off-a-bit-but-more-upside-likely.html
///
Metals story 1
* graphic-2019 asset returns: (adds official prices, adds dollar) zandi shabalala london, sept 17 (reuters) - copper edged lower tuesday investors doubted u.s-china talks week would lead speedy resolution long-running trade dispute recovery metals demand.
"the market fooled couple 

(kitco news) - group mining investors recommending mid-tier gold produces look merger acquisition opportunities unlock shareholder value.
report noted mid-tier multi-asset gold producers highest g&a compared senior gold produces, single asset mid-tier producer even non-gold mining companies.
figures suggest far many mid-tier, multi-asset gold producers demonstrate need consolidation space particularly." report comes mining sector seen renewed investor interest gold prices pushed six-year high.
Ссылка на статью: https://www.kitco.com/news/2019-09-16/Wall-Street-slips-at-open-after-Saudi-attacks-energy-stocks-surge.html
///
Metals story 34
editor's note: get caught minutes speedy summary today's must-read news stories expert opinions moved precious metals financial markets.
(kitco news) - kitco aficionados, time say think gold prices heading short-term.
kitco readers 9 a.m. edt, thursday cast vote.
readers also send comments newsfeedback@kitco.com could featured story.
share thoughts gol

(kitco news) - gold silver prices sharply daily highs early u.s. trading thursday.
bears' next near-term downside price breakout objective pushing december futures prices solid technical support week’s low $1,492.10.
wyckoff's market rating: 7.0 december silver futures bulls firm overall near-term technical advantage.
silver bulls' next upside price breakout objective closing prices solid technical resistance $19.00 ounce.
Ссылка на статью: https://www.kitco.com/news/video/show/Kitco-NEWS/2539/2019-09-12/Junior-miners-are-next-in-line-for-price-explosion-says-CEO#
///
